In [1]:
import teneva
import pysabr
import numpy as np
import time
import pandas as pd
import os

In [2]:
Ss = np.linspace(0.005, 0.07, num=3)
Ks = np.linspace(0.005, 0.07, num=3)
Ts = np.linspace(0.5, 20., num=3)
V_atms = np.linspace(0.1, 1.5, num=3) / 100 # revised
betas = np.linspace(0.1, 0.7, num=3)
rhos = np.linspace(-0.4, 0.4, num=3)
volvols = np.linspace(0.0001, 0.5, num=3)
displacements = np.linspace(0.0, 0.03, num=5)

In [3]:
data_path = "../data/"

In [4]:
if os.path.exists(data_path + "sample_lognormal_vol_revised.npy"):
    vols = np.load(data_path + "sample_lognormal_vol_revised.npy")

else:
    start = time.time()
    sabr_processes = [pysabr.Hagan2002LognormalSABR(S, d, T, V_atm, beta, rho, volvol) \
                      for S in Ss \
                      for T in Ts \
                      for V_atm in V_atms \
                      for beta in betas \
                      for rho in rhos \
                      for volvol in volvols \
                      for d in displacements
                     ]

    vols = np.reshape([sabr.lognormal_vol(k) for sabr in sabr_processes for k in Ks], \
                       (Ss.shape[0], Ts.shape[0], V_atms.shape[0], betas.shape[0], rhos.shape[0], \
                        volvols.shape[0], displacements.shape[0], Ks.shape[0])\
                     )

    end = time.time()
    print(f'compute vols time: {end - start}')
    os.makedirs("data_path")
    np.save(data_path + "sample_lognormal_vol_revised.npy", vols)

compute vols time: 55.1283757686615


In [5]:
print(vols.shape) # S, T, V_atm, Beta, Rho, Volvol, Displacement, K

(3, 3, 3, 3, 3, 3, 5, 3)


# Debug Below

In [6]:
from pysabr import black

In [7]:
names = ["S", "T", "V_atm", "Beta", "Rho", "Volvol", "Displacement", "K"]
multiindex = pd.MultiIndex.from_product([range(i) for i in vols.shape],
                                        names=names
                                       )
full_df = pd.DataFrame(vols.reshape((-1,1)), index=multiindex, columns=["Lognormal_vol"])
full_df.describe()

,Lognormal_vol
count,10935.000000
mean,0.344955
std,0.560070
min,-1.439675
25%,0.086765
50%,0.193700
75%,0.344270
max,4.034834


In [8]:
negative_vols = full_df.loc[(full_df < 0).values].copy()
negative_vols

Lognormal_vol
S T V_atm Beta Rho Volvol Displacement K               
0 2 1     2    0   2      0            1      -1.330587
                                       2      -1.439675
                          1            1      -0.614902
                                       2      -1.171503
    2     2    0   2      0            1      -1.277620
                                       2      -1.433733
                          1            1      -0.750018
                                       2      -1.238284
                          2            1      -0.243057
                                       2      -0.921582

In [9]:
test_inputs = negative_vols.drop(columns="Lognormal_vol").reset_index(drop=False)
test_inputs

,S,T,V_atm,Beta,Rho,Volvol,Displacement,K
0,0,2,1,2,0,2,0,1
1,0,2,1,2,0,2,0,2
2,0,2,1,2,0,2,1,1
3,0,2,1,2,0,2,1,2
4,0,2,2,2,0,2,0,1
5,0,2,2,2,0,2,0,2
6,0,2,2,2,0,2,1,1
7,0,2,2,2,0,2,1,2
8,0,2,2,2,0,2,2,1
9,0,2,2,2,0,2,2,2


In [10]:
test_inputs["S"] = test_inputs["S"].map(pd.Series(Ss))
test_inputs["T"] = test_inputs["T"].map(pd.Series(Ts))
test_inputs["V_atm"] = test_inputs["V_atm"].map(pd.Series(V_atms))
test_inputs["Beta"] = test_inputs["Beta"].map(pd.Series(betas))
test_inputs["Rho"] = test_inputs["Rho"].map(pd.Series(rhos))
test_inputs["Volvol"] = test_inputs["Volvol"].map(pd.Series(volvols))
test_inputs["Displacement"] = test_inputs["Displacement"].map(pd.Series(displacements))
test_inputs["K"] = test_inputs["K"].map(pd.Series(Ks))

test_inputs

,S,T,V_atm,Beta,Rho,Volvol,Displacement,K
0,0.005,20.0,0.008,0.7,-0.4,0.5,0.0000,0.0375
1,0.005,20.0,0.008,0.7,-0.4,0.5,0.0000,0.0700
2,0.005,20.0,0.008,0.7,-0.4,0.5,0.0075,0.0375
3,0.005,20.0,0.008,0.7,-0.4,0.5,0.0075,0.0700
4,0.005,20.0,0.015,0.7,-0.4,0.5,0.0000,0.0375
5,0.005,20.0,0.015,0.7,-0.4,0.5,0.0000,0.0700
6,0.005,20.0,0.015,0.7,-0.4,0.5,0.0075,0.0375
7,0.005,20.0,0.015,0.7,-0.4,0.5,0.0075,0.0700
8,0.005,20.0,0.015,0.7,-0.4,0.5,0.0150,0.0375
9,0.005,20.0,0.015,0.7,-0.4,0.5,0.0150,0.0700


In [11]:
"""
reference:
https://github.com/ynouri/pysabr/blob/master/pysabr/models/hagan_2002_lognormal_sabr.py
"""

def lognormal_vol(k, f, t, alpha, beta, rho, volvol, full_output=False):
    """
    Hagan's 2002 SABR lognormal vol expansion.
    The strike k can be a scalar or an array, the function will return an array
    of lognormal vols.
    """
    # Negative strikes or forwards
    if k <= 0 or f <= 0:
        return 0.
    eps = 1e-07
    logfk = np.log(f / k)
    fkbeta = (f*k)**(1 - beta)
    a = (1 - beta)**2 * alpha**2 / (24 * fkbeta)
    b = 0.25 * rho * beta * volvol * alpha / fkbeta**0.5
    c = (2 - 3*rho**2) * volvol**2 / 24
    d = fkbeta**0.5
    v = (1 - beta)**2 * logfk**2 / 24
    w = (1 - beta)**4 * logfk**4 / 1920
    z = volvol * fkbeta**0.5 * logfk / alpha
    
    if not full_output:
        
        # if |z| > eps
        if abs(z) > eps:
            vz = alpha * z * (1 + (a + b + c) * t) / (d * (1 + v + w) * _x(rho, z))
            return vz
        # if |z| <= eps
        else:
            v0 = alpha * (1 + (a + b + c) * t) / (d * (1 + v + w))
            return v0
        
    else:
        
        # if |z| > eps
        if abs(z) > eps:
            vz = alpha * z * (1 + (a + b + c) * t) / (d * (1 + v + w) * _x(rho, z))
            return_var = "vz"
            output_df = pd.DataFrame([[vz, return_var, z, _x(rho, z), z/_x(rho, z), (a + b + c) * t,
                                     a, b, c, d, v, w]],
                                    columns=["LOGNORMAL_VOL", "return_var", "z", "_x(rho, z)", "z/_x(rho, z)",
                                             "(a + b + c) * t", "a", "b", "c", "d", "v", "w"]
                                    )
            return output_df
        # if |z| <= eps
        else:
            v0 = alpha * (1 + (a + b + c) * t) / (d * (1 + v + w))
            return_var = "v0"
            output_df = pd.DataFrame([[v0, return_var, np.nan, np.nan, np.nan, (a + b + c) * t,
                                     a, b, c, d, v, w]],
                                    columns=["LOGNORMAL_VOL", "return_var", "z", "_x(rho, z)", "z/_x(rho, z)",
                                             "(a + b + c) * t", "a", "b", "c", "d", "v", "w"]
                                    )
            return output_df
        
    

def _x(rho, z):
    """Return function x used in Hagan's 2002 SABR lognormal vol expansion."""
    a = (1 - 2*rho*z + z**2)**.5 + z - rho
    b = 1 - rho
    return np.log(a / b)


def alpha_func(v_atm_ln, f, t, beta, rho, volvol):
    """    
    Compute SABR parameter alpha to an ATM lognormal volatility.
    Alpha is determined as the root of a 3rd degree polynomial. Return a single
    scalar alpha.
    """
    f_ = f ** (beta - 1)
    p = [
        t * f_**3 * (1 - beta)**2 / 24,
        t * f_**2 * rho * beta * volvol / 4,
        (1 + t * volvol**2 * (2 - 3*rho**2) / 24) * f_,
        -v_atm_ln
    ]
    roots = np.roots(p)
    roots_real = np.extract(np.isreal(roots), np.real(roots))
    # Note: the double real roots case is not tested
    alpha_first_guess = v_atm_ln * f**(1-beta)
    i_min = np.argmin(np.abs(roots_real - alpha_first_guess))
    return roots_real[i_min]

In [12]:
test_outputs = test_inputs.copy()
v_atm_slns = []
alphas = []
part_output_df = pd.DataFrame()

for [f, t, v_atm_n, beta, rho, volvol, s, k] in test_inputs.values:
    # Convert ATM normal vol to ATM shifted lognormal
    v_atm_sln = black.normal_to_shifted_lognormal(f, f, s, t, v_atm_n)
    alpha = alpha_func(v_atm_sln, f+s, t, beta, rho, volvol)
    part_output_df_i = lognormal_vol(k+s, f+s, t, alpha, beta, rho, volvol, True) 
    
    # save result
    v_atm_slns.append(v_atm_sln)
    alphas.append(alphas)
    part_output_df = pd.concat([part_output_df, part_output_df_i], ignore_index=True)

In [13]:
part_output_df

,LOGNORMAL_VOL,return_var,z,"_x(rho, z)","z/_x(rho, z)",(a + b + c) * t,a,b,c,d,v,w
0,-1.330587,vz,-0.179154,-0.184960,0.968614,-1.248020,0.118584,-0.196818,0.015833,0.276026,0.015224,0.000070
1,-1.439675,vz,-0.257681,-0.268983,0.957981,-1.301200,0.098334,-0.179228,0.015833,0.303117,0.026117,0.000205
2,-0.614902,vz,-0.100539,-0.102461,0.981241,-1.098977,0.152180,-0.222962,0.015833,0.325476,0.006153,0.000011
3,-1.171503,vz,-0.155373,-0.159811,0.972232,-1.207793,0.129280,-0.205503,0.015833,0.353128,0.012484,0.000047
4,-1.277620,vz,-0.176278,-0.181909,0.969042,-1.234220,0.122485,-0.200030,0.015833,0.276026,0.015224,0.000070
5,-1.433733,vz,-0.253543,-0.264524,0.958489,-1.294984,0.101570,-0.182152,0.015833,0.303117,0.026117,0.000205
6,-0.750018,vz,-0.102061,-0.104040,0.980978,-1.122588,0.147673,-0.219636,0.015833,0.325476,0.006153,0.000011
7,-1.238284,vz,-0.157726,-0.162292,0.971866,-1.223049,0.125451,-0.202437,0.015833,0.353128,0.012484,0.000047
8,-0.243057,vz,-0.073157,-0.074190,0.986073,-1.037500,0.163151,-0.230859,0.015833,0.357420,0.003493,0.000004
9,-0.921582,vz,-0.117903,-0.120519,0.978290,-1.154732,0.141192,-0.214762,0.015833,0.384209,0.007851,0.000018


In [14]:
negative_vols

Lognormal_vol
S T V_atm Beta Rho Volvol Displacement K               
0 2 1     2    0   2      0            1      -1.330587
                                       2      -1.439675
                          1            1      -0.614902
                                       2      -1.171503
    2     2    0   2      0            1      -1.277620
                                       2      -1.433733
                          1            1      -0.750018
                                       2      -1.238284
                          2            1      -0.243057
                                       2      -0.921582

In [15]:
# check the previous verifying calculation is right
np.allclose(part_output_df.LOGNORMAL_VOL.values, negative_vols.Lognormal_vol.values)

True

In [ ]:
output_df = pd.concat([pd.DataFrame({"v_atm_sln":v_atm_slns, "alpha":alphas}), part_output_df], axis=1)
output_df

# output_df = pd.DataFrame(np.hstack([np.array(v_atm_slns).reshape(-1,1), np.array(alphas).reshape(-1,1), part_output_df.values]),
#                         columns=["v_atm_sln", "alpha"] + part_output_df.columns.tolist())
# output_df

In [ ]:
merged_df = pd.concat([test_inputs, output_df], axis=1)
merged_df

# merged_df = pd.DataFrame(np.hstack([test_inputs.values, output_df.values]),
#                         columns=test_inputs.columns.tolist() + output_df.columns.tolist())
# merged_df

Observe the calculation of vz:  
$$v_z = alpha * z * \frac{1 + (a + b + c) * t}{d * (1 + v + w) * \_x(rho, z)}$$  
We can find that $alpha$, $\frac{z}{\_x(rho, z)}$ and $d$ are postive for all negative vols. $v$ and $w$ are positive by definition. It is $(a + b + c) * t < -1$ causes vol negative. And $a + b + c$ is not big actually. So if we shrink the maximum of $t$ to 15, we can avoid negative vols.